# Tasks : Topic Modelling
# Data Source : "https://www.gutenberg.org/files/67278/67278-0.txt"
# Data Type: Text
Title of Book : A secret Service : Being tale of Nihilist


# Step A : 
* Dowloading External Spacy Library to keep Specific tagger component(Noun, Verb,Adj,Adverb)
* Downloading PylDAvis to Visualize the generated topics 


In [ ]:
# Run in terminal or command prompt
!python3 -m spacy download en

     |████████████████████████████████| 12.0 MB 568 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install pyLDAvis

# Step B: Import Important packages
* The core packages needed in this tasks are re,nltk gensim, spacy numpy, to do text preprocessing ,and creating vectorization of text and the three models modules LDA,NMF,LSI

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from nltk.corpus import stopwords
import nltk; 
nltk.download('stopwords')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# STEP C : Data Extraction And Preprocessing
* urllib module was imported for the requested texted data from guterburb web link as shown
* Text data extracted was stored in a list for further preprocessing
# The following preprocessing was done to prepare the text data for topic modelling
* removal of emails and unwanted Ascii xters using remove_emails_nelinexters() function
* tokenization was done with UDF sent_to_words() function
* Stop words was removed from the data list using the stop_removal() function. non important words like "subject",'from', 're', 'edu', 'use' was added to be removed into the stop words corpus
* Finally Lemmatization was done in addition to allowing important taggers like NOUN.VB,ADJ,ADV using spacy module.

# The functions was triggered as highlighted in the order above to prepare the data. as shown as follws.


In [ ]:
import urllib
url = "https://www.gutenberg.org/files/67278/67278-0.txt"
file = urllib.request.urlopen(url)
data=[]
for line in file:
  decoded_line = line.decode("utf-8")
  data.append(decoded_line)
 

In [ ]:
data

['\ufeffThe Project Gutenberg eBook of A Secret Service: Being Strange Tales\r\n',
 'of a Nihilist, by William Le Queux\r\n',
 '\r\n',
 'This eBook is for the use of anyone anywhere in the United States and\r\n',
 'most other parts of the world at no cost and with almost no restrictions\r\n',
 'whatsoever. You may copy it, give it away or re-use it under the terms\r\n',
 'of the Project Gutenberg License included with this eBook or online at\r\n',
 'www.gutenberg.org. If you are not located in the United States, you\r\n',
 'will have to check the laws of the country where you are located before\r\n',
 'using this eBook.\r\n',
 '\r\n',
 'Title: A Secret Service: Being Strange Tales of a Nihilist\r\n',
 '\r\n',
 'Author: William Le Queux\r\n',
 '\r\n',
 'Release Date: January 29, 2022 [eBook #67278]\r\n',
 '\r\n',
 'Language: English\r\n',
 '\r\n',
 'Produced by: Carlos Colon, the University of California and the Online\r\n',
 '             Distributed Proofreading Team at https://www.pg

In [ ]:
#printing out some of the data text based on the list index.
print(data[2])
print('\n\n')
print(data[100])
print('\n\n')
print(data[300])






shivering humanity trudging through trackless snows and driven to their




military service until the day arrived when I was free to return and seek



In [ ]:
# pre-process the text data
#removing emails and unwanted xters text 
def remove_emails_newlinexters(dt):
  #remove emails
  dt = [re.sub('\S*@\S*\s?', '', sent) for sent in dt]
  # Remove new line characters
  dt = [re.sub('\s+', ' ', sent) for sent in dt]
  # Remove distracting single quotes
  dt = [re.sub("\'", "", sent) for sent in dt]
  return dt

#tokenization
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#stop-words removal
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [ ]:
#Function call to remove emails and unwanted xter
data=remove_emails_newlinexters(data)

In [ ]:
#tokenisation words
data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'project', 'gutenberg', 'ebook', 'of', 'secret', 'service', 'being', 'strange', 'tales']]


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [ ]:
print(data_lemmatized[:1])

[['project', 'secret', 'service', 'strange', 'tale']]


# STEP 4: Next , we import the scikit-learn packages to build our tune and build our models. 
* Bow and tfidf vectorization was used 
* Hyperparameter tuning carried out using Gridsearchcv to tune most important parameters like n_components which indicates number of topics and learning_decay
The best paameter was used in th model building.
* Model development was done for both vectorization methods.to ascertain the best performer. both vectorization methods with tuned parameters gave same results. i.e model complexity of both vectorization methods yield same results as shown.
* 


In [ ]:
# import packages from scikit-learn
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [ ]:
count_vectorizer = CountVectorizer( analyzer='word',
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
count_data_vectorized = count_vectorizer.fit_transform(data_lemmatized[0])

In [ ]:
# Initialize tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=0.05)
tfidf_data_vectorized=tfidf_vectorizer.fit_transform(data_lemmatized[0])

In [ ]:
# Define Search Param to hypertune most important parameter for LDA
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model_countvectorizer=model.fit(count_data_vectorized)
model_tfidf=model.fit(tfidf_data_vectorized)


In [ ]:
# Best Model
best_lda_model_count = model_countvectorizer.best_estimator_
best_lda_model_tfidf = model_tfidf.best_estimator_

# Model Parameters
print("Best Model's Params: ", model_countvectorizer.best_params_)
print("Best Model's Params: ", model_tfidf.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model_countvectorizer.best_score_)
print("Best Log Likelihood Score: ", model_tfidf.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model_count.perplexity(count_data_vectorized))
print("Model Perplexity: ", best_lda_model_tfidf.perplexity(tfidf_data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -17.433659615019373
Best Log Likelihood Score:  -17.433659615019373
Model Perplexity:  49.999995780269366
Model Perplexity:  49.999995780269366


In [ ]:
# Build a Latent Dirichlet Allocation Model
lda_model_count = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online',learning_decay=0.9) # no of components = no. of topics
lda_model_tfidf = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online',learning_decay=0.9) # no of components = no. of topics

lda_Z_count = lda_model_count.fit_transform(count_data_vectorized)
print(lda_Z_count.shape)  # (NO_DOCUMENTS, NO_TOPICS)
lda_Z_tfidf = lda_model_tfidf.fit_transform(tfidf_data_vectorized)
print(lda_Z_tfidf.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model_count = NMF(n_components= 10)
nmf_Z_count = nmf_model_count.fit_transform(count_data_vectorized)
print(nmf_Z_count.shape)  # (NO_DOCUMENTS, NO_TOPICS)

nmf_model_tfidf = NMF(n_components= 10)
nmf_Z_tfidf = nmf_model_tfidf.fit_transform(tfidf_data_vectorized)
print(nmf_Z_tfidf.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model_count = TruncatedSVD(n_components=4)
lsi_Z_count = lsi_model_count.fit_transform(count_data_vectorized)
print(lsi_Z_count.shape)  # (NO_DOCUMENTS, NO_TOPICS)

# Build a Latent Semantic Indexing Model
lsi_model_tfidf = TruncatedSVD(n_components=4)
lsi_Z_tfidf = lsi_model_tfidf.fit_transform(tfidf_data_vectorized)
print(lsi_Z_tfidf.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(5, 10)
(5, 10)
(5, 10)
(5, 10)
(5, 4)
(5, 4)


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


# Step 5: Inspecting inferred topics.
* This section shows the inspection of generated topics with feature names and metrics using print_topics() functions

In [ ]:
# inspect the inferred topics
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        
'''# inspect the inferred topics
def print_topics(model, tfidf_vectorizer, top_n=10):
    for idx, topic in enumerate(model_tfidf.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])'''

'# inspect the inferred topics\ndef print_topics(model, tfidf_vectorizer, top_n=10):\n    for idx, topic in enumerate(model_tfidf.components_):\n        print("Topic %d:" % (idx))\n        print([(tfidf_vectorizer.get_feature_names()[i], topic[i])\n                        for i in topic.argsort()[:-top_n - 1:-1]])'

In [ ]:
print("LDA Model For CountVectorizer:")
print_topics(lda_model_count, count_vectorizer)
print("=" * 20)

print("LDA Model For tfidf Vectorizer:")
print_topics(lda_model_tfidf, tfidf_vectorizer)
print("=" * 20)

LDA Model For CountVectorizer:
Topic 0:
[('strange', 0.5511831437573891), ('service', 0.4858163764529976), ('project', 0.4626373692514554), ('tale', 0.4580019199798155), ('secret', 0.38614970720719083)]
Topic 1:
[('project', 1.0765734034287975), ('secret', 0.4909211317825914), ('strange', 0.48959446319899885), ('service', 0.48408829338942605), ('tale', 0.38813155878941424)]
Topic 2:
[('strange', 0.48413496630609404), ('tale', 0.46032839950035875), ('secret', 0.4583932335419078), ('service', 0.4515144436986946), ('project', 0.42986964026116387)]
Topic 3:
[('secret', 1.0098132460168867), ('service', 0.48439664641883645), ('strange', 0.457637825290475), ('project', 0.42956636590314556), ('tale', 0.4258728507118061)]
Topic 4:
[('service', 0.4836180672312318), ('project', 0.48113498948452676), ('strange', 0.45981867764896245), ('tale', 0.4555953850308754), ('secret', 0.44754130875530673)]
Topic 5:
[('service', 0.5056902211233659), ('strange', 0.4822479084622233), ('project', 0.4508425257467

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("NMF Model for Count Vectorizer:")
print_topics(nmf_model_count, count_vectorizer)
print("=" * 20)

print("NMF Model for tfidf vectorizer:")
print_topics(nmf_model_tfidf, tfidf_vectorizer)
print("=" * 20)

NMF Model for Count Vectorizer:
Topic 0:
[('project', 0.20097542882636912), ('tale', 9.62964972193618e-35), ('strange', 0.0), ('service', 0.0), ('secret', 0.0)]
Topic 1:
[('service', 0.422773986885542), ('tale', 0.0), ('strange', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 2:
[('secret', 0.4111909034114774), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('project', 0.0)]
Topic 3:
[('project', 0.44775531459333934), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('secret', 0.0)]
Topic 4:
[('tale', 1.5883425171547372), ('strange', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 5:
[('service', 0.20465157892190855), ('tale', 0.0), ('strange', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 6:
[('tale', 1.5547630261356795), ('strange', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 7:
[('strange', 0.6353941378528275), ('tale', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 8:
[('tale', 0.33329815218397857), ('strange', 0.0), ('

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("LSI Model for Count Vectorizer:")
print_topics(lsi_model_count, count_vectorizer)
print("=" * 20)

print("LSI Model for tfidf vectorizer:")
print_topics(lsi_model_tfidf, tfidf_vectorizer)
print("=" * 20)

LSI Model for Count Vectorizer:
Topic 0:
[('tale', 0.9626509401538991), ('strange', 0.1194558443571551), ('project', -0.0), ('secret', -0.1165493505195066), ('service', -0.21318939364615405)]
Topic 1:
[('service', 0.8681544899437175), ('secret', 0.4326607448301696), ('tale', 0.24253562503633297), ('strange', 0.016998001700269105), ('project', -0.0)]
Topic 2:
[('secret', 0.8919529754965996), ('strange', 0.07432941462471662), ('tale', -0.0), ('project', -0.0), ('service', -0.4459764877482998)]
Topic 3:
[('strange', 0.9899073225856367), ('service', 0.04430616664799511), ('project', -0.0), ('secret', -0.06033919355813885), ('tale', -0.12033136751923725)]
LSI Model for tfidf vectorizer:
Topic 0:
[('service', 0.6520944686386168), ('tale', 0.491368343361388), ('project', -0.0), ('strange', -0.28632520137648626), ('secret', -0.5013460223936359)]
Topic 1:
[('strange', 0.6991050010351759), ('tale', 0.1717830611428392), ('project', 0.0), ('service', -0.3026834937222956), ('secret', -0.62460025621

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Step 6: pylDAvis in Action
* Displaying resultant model result using pylDAvis visualization tools.

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_count, count_data_vectorized, count_vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  14.999539
1       0.648955  71.707100       2        1  14.999530
9      71.019684   9.971006       3        1  14.999448
3       3.785981  33.185135       4        1  14.999414
6     -42.666500  -9.222070       5        1  14.999373
0      -3.621689 -43.526516       6        1   5.000560
4      -3.632402  -4.171708       7        1   5.000547
5     -36.584400  35.804413       8        1   5.000543
2      44.391380  50.615383       9        1   5.000536
8      40.235809 -31.591665      10        1   5.000512, topic_info=      Term      Freq     Total Category  logprob  loglift
2  service  1.000000  1.000000  Default   5.0000   5.0000
3  strange  1.000000  1.000000  Default   4.0000   4.0000
0  project  0.000000  0.000000  Default   3.0000   3.0000
4     tale  0.000000  0.000000  Default   2.0000   2.0000
1   secret  1.000000  1.000000  Default   1.0000   1.0000
4     tale  0.273078  0.963364   Topic1  -1.0103   0.6365
2  service  0.130118  1.053150   Topic1  -1.7516  -0.1939
1   secret  0.119425  1.000852   Topic1  -1.8374  -0.2288
0  project  0.113253  0.962894   Topic1  -1.8904  -0.2432
3  strange  0.114102  1.019741   Topic1  -1.8829  -0.2931
0  project  0.275630  0.962894   Topic2  -1.0010   0.6463
1   secret  0.125688  1.000852   Topic2  -1.7862  -0.1777
3  strange  0.125348  1.019741   Topic2  -1.7889  -0.1991
2  service  0.123939  1.053150   Topic2  -1.8003  -0.2426
4     tale  0.099371  0.963364   Topic2  -2.0212  -0.3744
2  service  0.301616  1.053150   Topic3  -0.9109   0.6468
3  strange  0.120668  1.019741   Topic3  -1.8270  -0.2371
4     tale  0.112558  0.963364   Topic3  -1.8966  -0.2498
1   secret  0.110114  1.000852   Topic3  -1.9185  -0.3099
0  project  0.105016  0.962894   Topic3  -1.9659  -0.3187
1   secret  0.269773  1.000852   Topic4  -1.0225   0.5861
2  service  0.129407  1.053150   Topic4  -1.7571  -0.1994
3  strange  0.122259  1.019741   Topic4  -1.8139  -0.2240
0  project  0.114759  0.962894   Topic4  -1.8772  -0.2299
4     tale  0.113773  0.963364   Topic4  -1.8858  -0.2391
3  strange  0.277675  1.019741   Topic5  -0.9936   0.5963
1   secret  0.139201  1.000852   Topic5  -1.6841  -0.0755
4     tale  0.118772  0.963364   Topic5  -1.8428  -0.1961
0  project  0.108108  0.962894   Topic5  -1.9369  -0.2897
2  service  0.106213  1.053150   Topic5  -1.9546  -0.3969
3  strange  0.058798  1.019741   Topic6  -1.4475   0.1424
0  project  0.049353  0.962894   Topic6  -1.6226   0.0247
4     tale  0.048858  0.963364   Topic6  -1.6327   0.0141
2  service  0.051825  1.053150   Topic6  -1.5737  -0.0160
1   secret  0.041193  1.000852   Topic6  -1.8033  -0.1947
0  project  0.051680  0.962894   Topic7  -1.5765   0.0708
4     tale  0.048937  0.963364   Topic7  -1.6310   0.0157
2  service  0.051947  1.053150   Topic7  -1.5713  -0.0137
3  strange  0.049391  1.019741   Topic7  -1.6218  -0.0319
1   secret  0.048072  1.000852   Topic7  -1.6489  -0.0403
2  service  0.054543  1.053150   Topic8  -1.5226   0.0351
3  strange  0.052015  1.019741   Topic8  -1.5700   0.0198
0  project  0.048627  0.962894   Topic8  -1.6374   0.0099
4     tale  0.047012  0.963364   Topic8  -1.6712  -0.0244
1   secret  0.047830  1.000852   Topic8  -1.6539  -0.0453
4     tale  0.050386  0.963364   Topic9  -1.6018   0.0449
3  strange  0.052992  1.019741   Topic9  -1.5514   0.0385
1   secret  0.050174  1.000852   Topic9  -1.6061   0.0025
0  project  0.047052  0.962894   Topic9  -1.6703  -0.0231
2  service  0.049422  1.053150   Topic9  -1.6212  -0.0635
4     tale  0.050619  0.963364  Topic10  -1.5972   0.0495
2  service  0.054119  1.053150  Topic10  -1.5304   0.0273
0  project  0.049415  0.962894  Topic10  -1.6213   0.0259
1   secret  0.049381  1.000852  Topic10  -1.6220  -0.0134
3  strange  0.046491  1.019741  Topic10  -1.6823  -0.0924, token_table=Empty DataFrame
Columns:

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_tfidf, tfidf_data_vectorized, tfidf_vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  14.999495
0       0.648955  71.707100       2        1  14.999487
2      71.019684   9.971006       3        1  14.999458
9       3.785981  33.185135       4        1  14.999441
5     -42.666500  -9.222070       5        1  14.999417
4      -3.621689 -43.526516       6        1   5.000577
7      -3.632402  -4.171708       7        1   5.000553
3     -36.584400  35.804413       8        1   5.000547
8      44.391380  50.615383       9        1   5.000528
1      40.235809 -31.591665      10        1   5.000498, topic_info=      Term      Freq     Total Category  logprob  loglift
4     tale  0.000000  0.000000  Default   5.0000   5.0000
1   secret  1.000000  1.000000  Default   4.0000   4.0000
2  service  1.000000  1.000000  Default   3.0000   3.0000
3  strange  0.000000  0.000000  Default   2.0000   2.0000
0  project  1.000000  1.000000  Default   1.0000   1.0000
0  project  0.281493  1.004656   Topic1  -0.9799   0.6249
1   secret  0.128071  1.007103   Topic1  -1.7675  -0.1651
2  service  0.120404  1.003548   Topic1  -1.8292  -0.2233
3  strange  0.113330  0.992496   Topic1  -1.8897  -0.2728
4     tale  0.106677  0.992198   Topic1  -1.9502  -0.3330
2  service  0.288507  1.003548   Topic2  -0.9553   0.6506
3  strange  0.121808  0.992496   Topic2  -1.8176  -0.2006
0  project  0.118969  1.004656   Topic2  -1.8412  -0.2364
4     tale  0.116275  0.992198   Topic2  -1.8641  -0.2468
1   secret  0.104415  1.007103   Topic2  -1.9717  -0.3693
4     tale  0.280009  0.992198   Topic3  -0.9852   0.6321
3  strange  0.125313  0.992496   Topic3  -1.7892  -0.1723
2  service  0.118896  1.003548   Topic3  -1.8418  -0.2359
0  project  0.115186  1.004656   Topic3  -1.8735  -0.2687
1   secret  0.110569  1.007103   Topic3  -1.9144  -0.3120
3  strange  0.270674  0.992496   Topic4  -1.0191   0.5979
0  project  0.123900  1.004656   Topic4  -1.8006  -0.1958
1   secret  0.122277  1.007103   Topic4  -1.8137  -0.2114
4     tale  0.120264  0.992198   Topic4  -1.8303  -0.2131
2  service  0.112857  1.003548   Topic4  -1.8939  -0.2880
1   secret  0.284861  1.007103   Topic5  -0.9680   0.6343
0  project  0.127158  1.004656   Topic5  -1.7746  -0.1698
3  strange  0.118559  0.992496   Topic5  -1.8446  -0.2277
4     tale  0.111029  0.992198   Topic5  -1.9102  -0.2930
2  service  0.108365  1.003548   Topic5  -1.9345  -0.3286
1   secret  0.056966  1.007103   Topic6  -1.4791   0.1232
4     tale  0.050948  0.992198   Topic6  -1.5908   0.0265
2  service  0.050926  1.003548   Topic6  -1.5912   0.0147
3  strange  0.047904  0.992496   Topic6  -1.6524  -0.0354
0  project  0.043285  1.004656   Topic6  -1.7538  -0.1490
4     tale  0.053746  0.992198   Topic7  -1.5373   0.0800
1   secret  0.053211  1.007103   Topic7  -1.5473   0.0551
2  service  0.048692  1.003548   Topic7  -1.6360  -0.0302
3  strange  0.047912  0.992496   Topic7  -1.6522  -0.0352
0  project  0.046466  1.004656   Topic7  -1.6829  -0.0781
4     tale  0.053326  0.992198   Topic8  -1.5451   0.0721
2  service  0.052859  1.003548   Topic8  -1.5539   0.0519
3  strange  0.049410  0.992496   Topic8  -1.6214  -0.0045
1   secret  0.048722  1.007103   Topic8  -1.6354  -0.0331
0  project  0.045711  1.004656   Topic8  -1.6992  -0.0944
2  service  0.055904  1.003548   Topic9  -1.4979   0.1080
1   secret  0.051043  1.007103   Topic9  -1.5889   0.0135
3  strange  0.049458  0.992496   Topic9  -1.6204  -0.0035
4     tale  0.049403  0.992198   Topic9  -1.6215  -0.0043
0  project  0.044217  1.004656   Topic9  -1.7324  -0.1277
0  project  0.058272  1.004656  Topic10  -1.4564   0.1484
4     tale  0.050519  0.992198  Topic10  -1.5992   0.0181
3  strange  0.048128  0.992496  Topic10  -1.6477  -0.0307
1   secret  0.046967  1.007103  Topic10  -1.6721  -0.0698
2  service  0.046138  1.003548  Topic10  -1.6899  -0.0840, token_table=Empty DataFrame
Columns:

## (4) How many Topics are there? 5 : tale,secret,service,strange,project
## (5) Both Vectoriztion method performed well with same log-likelihood and Model complexity as follows:

* Best Log Likelihood Score for countVectorizer:  -17.433659615019373
* Best Log Likelihood Score for tfidf:  -17.433659615019373
* Model Perplexity for countvectorizer:  49.999995780269366
* Model Perplexity tfidf:  49.999995780269366
## (6) 
## (7) hyperparameter tuning with GridSearchcv
* Best Model's Params for Count:  {'learning_decay': 0.7, 'n_components': 10}
* Best Model's Params for tfidf:  {'learning_decay': 0.7, 'n_components': 10}
